# Age Prediction Using 2D CNN

In [9]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses, callbacks
import numpy as np
import datetime

import scipy.io

In [5]:
url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"

dataset = tf.keras.utils.get_file(
    "wiki_crop", url, 
    untar=True, cache_dir='',
    cache_subdir=''
)
dataset_dir = os.path.join(os.path.dirname(dataset), 'wiki_crop')

811315200/811315200 ━━━━━━━━━━━━━━━━━━━━ 2739s 3us/step


In [10]:
mat = scipy.io.loadmat(os.path.join(dataset_dir, 'wiki.mat'))
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 16 16:25:20 2016',
 '__version__': '1.0',
 '__globals__': [],
 'wiki': array([[(array([[723671, 703186, 711677, ..., 720620, 723893, 713846]], dtype=int32), array([[2009, 1964, 2008, ..., 2013, 2011, 2008]], dtype=uint16), array([[array(['17/10000217_1981-05-05_2009.jpg'], dtype='<U31'),
                 array(['48/10000548_1925-04-04_1964.jpg'], dtype='<U31'),
                 array(['12/100012_1948-07-03_2008.jpg'], dtype='<U29'), ...,
                 array(['09/9998109_1972-12-27_2013.jpg'], dtype='<U30'),
                 array(['00/9999400_1981-12-13_2011.jpg'], dtype='<U30'),
                 array(['80/999980_1954-06-11_2008.jpg'], dtype='<U29')]],
               dtype=object), array([[1., 1., 1., ..., 1., 1., 0.]]), array([[array(['Sami Jauhojärvi'], dtype='<U15'),
                 array(['Dettmar Cramer'], dtype='<U14'),
                 array(['Marc Okrand'], dtype='<U11'), ...,
                 arr

In [12]:
mat['wiki']['dob'][0][0][0]

array([723671, 703186, 711677, ..., 720620, 723893, 713846], dtype=int32)

In [14]:
dob = np.vectorize(lambda x: datetime.datetime.fromordinal(x).year)(mat['wiki']['dob'][0][0][0])
photo_taken = mat["wiki"]["photo_taken"][0][0][0]

age = (photo_taken - dob).astype(np.float32)
age

array([27., 38., 59., ..., 40., 29., 53.], dtype=float32)

In [15]:
file_path = np.vectorize(lambda x: os.path.join(dataset_dir, x[0]))(mat['wiki']['full_path'][0][0][0])
file_path

array(['/tmp/.keras/wiki_crop/17/10000217_1981-05-05_2009.jpg',
       '/tmp/.keras/wiki_crop/48/10000548_1925-04-04_1964.jpg',
       '/tmp/.keras/wiki_crop/12/100012_1948-07-03_2008.jpg', ...,
       '/tmp/.keras/wiki_crop/09/9998109_1972-12-27_2013.jpg',
       '/tmp/.keras/wiki_crop/00/9999400_1981-12-13_2011.jpg',
       '/tmp/.keras/wiki_crop/80/999980_1954-06-11_2008.jpg'],
      dtype='<U59')

In [16]:
file_age_ds = tf.data.Dataset.from_tensor_slices((file_path, age))

2024-04-03 19:41:05.659735: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 19:41:05.660280: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
def parse_file(filename, label):
    img = tf.io.read_file(filename)
    img = tf.io.decode_jpeg(img, channels=1)
    img = tf.image.resize(img,[256, 256])
    return img, tf.expand_dims(label,0)

In [18]:
image_age_ds = file_age_ds.map(parse_file).shuffle(seed=2, buffer_size=64)
image_age_ds

<_ShuffleDataset element_spec=(TensorSpec(shape=(256, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

In [19]:
dataset_size = image_age_ds.cardinality().numpy()
batch_size = 32

AUTOTUNE = tf.data.AUTOTUNE
train_ds = image_age_ds.take(dataset_size*.6).batch(batch_size).prefetch(AUTOTUNE)
val_ds = image_age_ds.skip(dataset_size*.6).take(dataset_size*.2).batch(batch_size).prefetch(AUTOTUNE)
test_ds = image_age_ds.skip(dataset_size*.8).take(dataset_size*.2).batch(batch_size).prefetch(AUTOTUNE)

In [25]:
model = Sequential([
    layers.Input((256, 256,1)),
   
    layers.Conv2D(32, (7, 7), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((4,4), strides=4),
   
    layers.Conv2D(64,(3,3), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((4,4), strides=4),
    
    layers.Conv2D(128,(3,3), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((2,2), strides=2),

    layers.Conv2D(256,(1,1), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((2, 2), strides=2),


    layers.Flatten(),
    layers.Dense(64), 
    layers.ReLU(),
    layers.Dense(1)

])

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 250, 250, 32)   │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 250, 250, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 60, 60, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 13, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 6, 6, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_9 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 274,561 (1.05 MB)

 Trainable params: 274,561 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(
    optimizer="adam",
    loss=losses.MeanAbsoluteError(),
    metrics=["MAE"]
)

In [28]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[
        callbacks.TensorBoard(log_dir="logs/ex5")
    ]
)

Epoch 1/10


2024-04-03 19:57:31.843017: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.
2024-04-03 19:57:32.211229: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.


   1/1169 ━━━━━━━━━━━━━━━━━━━━ 46:50 2s/step - MAE: 35.2928 - loss: 35.2928

2024-04-03 19:57:32.775658: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.
2024-04-03 19:57:33.094440: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.


   2/1169 ━━━━━━━━━━━━━━━━━━━━ 19:17 992ms/step - MAE: 30.5824 - loss: 30.5824

2024-04-03 19:57:33.765547: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.


 599/1169 ━━━━━━━━━━━━━━━━━━━━ 6:56 731ms/step - MAE: 15.8281 - loss: 15.8281

KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
model.evaluate(test_ds)